In [ ]:
import numpy as np

import pandas as pd

import matplotlib.pyplot as plt

from pathlib import Path

import seaborn as sns

from collections import defaultdict

import pickle

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from torch.utils.data import TensorDataset

import torchvision.datasets as dset
import torchvision.transforms as T

In [ ]:
USE_GPU = True
dtype = torch.float32 # We will be using float throughout this tutorial.

if USE_GPU and torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

# Constant to control how frequently we print train loss.
print_every = 100
print('using device:', device)

In [ ]:

# data = pd.read_csv('peaks_all.csv')
# print(data.head)
# print(data.shape)
# row_0 = np.array(data.columns)[1:]
# print(len(row_0))
# data_t = data.T
# print(data_t.shape)
# data_t = data_t.to_numpy()
# x = data_t[1:,:]
# with open('x.pickle', 'wb') as f:
#     pickle.dump(x,f)

In [ ]:
with open('x.pickle','rb') as f:
    x = torch.Tensor(pickle.load(f).astype(float))
print(x.shape)

In [ ]:
meta_data = pd.read_csv('metadata_seurat_onelayer.csv')
y = np.array(meta_data.loc[:,"Celltype"])

In [ ]:
y_set = set(y)
print(len(y_set))
labels_dict = defaultdict(int)
label_num = 0
for item in y_set:
    labels_dict[item] = label_num
    label_num += 1

labels = []
for item in y:
    labels.append(labels_dict[item])
labels = np.array(labels)
labels = torch.from_numpy(labels)

In [ ]:
TOTAL_DATAPOINTS = 14322
NUM_TRAIN = 12890
NUM_VAL = 716
NUM_TEST = 716

labels = torch.Tensor(labels)
dataset = TensorDataset(x,labels)

In [ ]:
# this function will automatically randomly split your dataset but you could also implement the split yourself
train_set, val_set, test_set = torch.utils.data.random_split(dataset, [NUM_TRAIN, NUM_TEST, NUM_VAL])

loader_train = DataLoader(train_set, batch_size=64)
loader_val = DataLoader(val_set, batch_size=64)

loader_test = DataLoader(test_set, batch_size=64)

In [ ]:
def train(model, optimizer, epochs=1, start_epoch=0):
    model = model.to(device=device)  # move the model parameters to CPU/GPU
    loss_list = []
    val_acc_list = []
    train_acc_list = []
    for e in range(epochs):
        print(f"epoch: {e+1}/{epochs}")
        for t, (x, y) in enumerate(loader_train):
            #x = x.unsqueeze(dim = 1)
         
            model.train()  # put model to training mode
            x = x.to(device=device, dtype=dtype)  # move to device, e.g. GPU
            y = y.to(device=device, dtype=torch.long)
        
            scores = model(x)
            loss = F.cross_entropy(scores, y)

            # Zero out all of the gradients for the variables which the optimizer
            # will update.
            optimizer.zero_grad()

            # This is the backwards pass: compute the gradient of the loss with
            # respect to each  parameter of the model.
            loss.backward()

            # Actually update the parameters of the model using the gradients
            # computed by the backwards pass.
            optimizer.step()

            if t % print_every == 0:
                print('Iteration %d, loss = %.4f' % (t, loss.item()))
                loss_list.append(loss.item())
                print("Train Accuracy:")
                train_acc = check_accuracy(loader_train, model)
                print("Validation Accuracy:")
                val_acc = check_accuracy(loader_val, model)
                train_acc_list.append(train_acc)
                val_acc_list.append(val_acc)
                np.savetxt("train.txt", np.array(train_acc_list))
                np.savetxt("val_acc.txt", np.array(val_acc_list))
                np.savetxt("losslist.txt", np.array(loss_list))
                print()

        if e % 20 == 0:
            torch.save(model.state_dict(), f'model_weights_orig_{e}.pth')
            print("model saved")
                

    return loss_list, train_acc_list, val_acc_list

def train_conv(model, optimizer, epochs=1, start_epoch=0):
    model = model.to(device=device)  # move the model parameters to CPU/GPU
    loss_list = []
    val_acc_list = []
    train_acc_list = []
    for e in range(epochs):
        print(f"epoch: {e+1}/{epochs}")
        for t, (x, y) in enumerate(loader_train):
            #x = x.unsqueeze(dim = 1)
         
            model.train()  # put model to training mode
            x = x.to(device=device, dtype=dtype)  # move to device, e.g. GPU
            y = y.to(device=device, dtype=torch.long)
        
            scores = model(x)
            loss = F.cross_entropy(scores, y)

            # Zero out all of the gradients for the variables which the optimizer
            # will update.
            optimizer.zero_grad()

            # This is the backwards pass: compute the gradient of the loss with
            # respect to each  parameter of the model.
            loss.backward()

            # Actually update the parameters of the model using the gradients
            # computed by the backwards pass.
            optimizer.step()

            if t % print_every == 0:
                print('Iteration %d, loss = %.4f' % (t, loss.item()))
                loss_list.append(loss.item())
                print("Train Accuracy:")
                train_acc = check_accuracy(loader_train, model)
                print("Validation Accuracy:")
                val_acc = check_accuracy(loader_val, model)
                train_acc_list.append(train_acc)
                val_acc_list.append(val_acc)
                np.savetxt("train.txt", np.array(train_acc_list))
                np.savetxt("val_acc.txt", np.array(val_acc_list))
                np.savetxt("losslist.txt", np.array(loss_list))
                print()

        if e % 20 == 0:
            torch.save(model.state_dict(), f'model_weights_orig_conv_{e}.pth')
            print("model saved")
                

    return loss_list, train_acc_list, val_acc_list
            
def check_accuracy(loader, model):
    num_correct = 0
    num_samples = 0
    model.eval()  # set model to evaluation mode
    with torch.no_grad():
        for x, y in loader:
            # x = x.unsqueeze(dim = 1)
            x = x.to(device=device, dtype=dtype)  # move to device, e.g. GPU
            y = y.to(device=device, dtype=torch.long)
            scores = model(x)
            preds = torch.argmax(scores,dim=1)
            num_correct += (preds == y).sum()
            num_samples += preds.size(0)
        acc = float(num_correct) / num_samples
        print('Got %d / %d correct (%.2f)' % (num_correct, num_samples, 100 * acc))
    return acc

def check_accuracy_conv(loader, model):
    num_correct = 0
    num_samples = 0
    model.eval()  # set model to evaluation mode
    with torch.no_grad():
        for x, y in loader:
            x = x.unsqueeze(dim = 1)
            x = x.to(device=device, dtype=dtype)  # move to device, e.g. GPU
            y = y.to(device=device, dtype=torch.long)
            scores = model(x)
            preds = torch.argmax(scores,dim=1)
            num_correct += (preds == y).sum()
            num_samples += preds.size(0)
        acc = float(num_correct) / num_samples
        print('Got %d / %d correct (%.2f)' % (num_correct, num_samples, 100 * acc))
    return acc

In [ ]:

logistic_regression = nn.Sequential(
    nn.Linear(187541, 22),
)

learning_rate = 5e-5
optimizer = optim.SGD(logistic_regression.parameters(), lr=learning_rate, momentum=0.9, nesterov=True)


conv_model = nn.Sequential(
    nn.Conv1d(1, 8, 9, padding = 'same'),
    nn.ReLU(),
    nn.MaxPool1d(2, stride=2),
    nn.Conv1d(8,16,9, padding = 'same'),
    nn.ReLU(),
    nn.MaxPool1d(2, stride=2),
    nn.Conv1d(16,32,9,  padding = 'same'),
    nn.ReLU(),
    nn.MaxPool1d(2, stride=2),
    nn.Conv1d(32,32,9,  padding = 'same'),
    nn.ReLU(),
    nn.MaxPool1d(2, stride=2),
    nn.Conv1d(32,32,9,  padding = 'same'),
    nn.ReLU(),
    nn.MaxPool1d(2, stride=2),
    nn.Flatten(),
    nn.Linear(64 * 2930, 22)
)


In [ ]:
# Start training
loss_list, train_acc_list, val_acc_list = train(logistic_regression, optimizer, epochs=100)


In [ ]:
train_acc_list, val_acc_list = list(np.array(train_acc_list)), list(np.array(val_acc_list))


In [ ]:
logistic_checkpoint = torch.load('model_weights_orig_60.pth')
conv_checkpoint = torch.load('model_weights_orig_conv.pth')

logistic_regression.load_state_dict(logistic_checkpoint)
logistic_regression.cuda()

conv_model.load_state_dict(conv_checkpoint)
conv_model.cuda()

In [ ]:
def saliency(seq, model):
    #we don't need gradients w.r.t. weights for a trained model
    for param in model.parameters():
        param.requires_grad = False
    
    #set model in eval mode
    model = model.to(device=device)
    model.eval()
    #transoform input PIL image to torch.Tensor and normalize
    # input = T(img)
    # input.unsqueeze_(0)

    #we want to calculate gradient of higest score w.r.t. input
    #so set requires_grad to True for input 
    seq.to(device=device)  
    seq.requires_grad = True
    #forward pass to calculate predictions
    preds = model(seq)
    score = torch.max(preds)
    #backward pass to get gradients of score predicted class w.r.t. input image
    score.to(device=device)  

    score.backward()
    #get max along channel axis
    saliency_map = torch.abs(seq.grad)
    #normalize to [0..1]
    saliency_map = (saliency_map - saliency_map.min())/(saliency_map.max()-saliency_map.min())

    # #apply inverse transform on image
    # with torch.no_grad():
    #     input_img = seq[0]


    input_vector = seq.cpu().detach().numpy()
    saliency_map = saliency_map.cpu().detach().numpy()

    plt.clf()
    saliency_map = np.reshape(saliency_map,(1,-1))
    plt.figure(figsize=(4,25))
    sns.heatmap(saliency_map,annot=True,cmap='viridis',cbar=True)
    plt.show()

In [ ]:
example_1 = x[1]
example_1 = example_1.cuda()

example_2 = x[2]
example_2 = example_2.cuda()

example_6 = x[6]
example_6 = example_6.cuda()

print(y[1])
print(y[2])
print(y[6])

In [ ]:
saliency(example_1, logistic_regression)

In [ ]:
saliency(example_2, logistic_regression)

In [ ]:
saliency(example_6, logistic_regression)

In [ ]:
check_accuracy(loader_test, logistic_regression)

In [ ]:
check_accuracy_conv(loader_val, conv_model)

In [ ]:
def data_analysis(loader, model):
    model.eval()  
    preds_list = []
    y_list = []
    for x, y in loader:
        x = x.to(device=device, dtype=dtype)  # move to device, e.g. GPU
        y = y.to(device=device, dtype=torch.long)
        scores = model(x)
        preds = torch.argmax(scores,dim=1)
        preds_list += preds
        y_list += y 
    return preds_list, y_list

In [ ]:
inverted_labels_dict = defaultdict(str)
for key,value in labels_dict.items():
    inverted_labels_dict[value] = key

In [ ]:
preds_list, y_list = data_analysis(loader_train, logistic_regression)
preds_list = [x.item() for x in preds_list]
y_list = [x.item() for x in y_list]
classes = set(y_list)
totals_d = defaultdict(int)
d = defaultdict(int)
for pred, y in zip(preds_list,y_list):
    totals_d[inverted_labels_dict[y]] += 1
    if pred != y:
        d[inverted_labels_dict[y]] += 1

percent_d = defaultdict(float)
for key,value in totals_d.items():
    percent_d[key] = d[key]/value

plt.bar(*zip(*percent_d.items()))
plt.xticks(rotation=90)
plt.show() 




In [ ]:
plt.bar(*zip(*totals_d.items()))
plt.xticks(rotation=90)
plt.show() 

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(10, 4))
# val_acc_list = np.loadtxt('val_acc.txt')
# train_acc_list = np.loadtxt('train.txt')
# loss_list = np.loadtxt('losslist.txt')

print(len(val_acc_list))
x_loss = [i for i in range(len(loss_list))]
x_acc = [i for i in range(len(val_acc_list))]

x_acc = x_acc[1:]

axs[0].plot(x_loss[2:], loss_list[2:])
axs[0].set_title('Loss vs Iteration')
axs[0].set_xlabel('Iteration')
axs[0].set_ylabel('Loss')

axs[1].plot(x_acc, val_acc_list[1:], label='val')
axs[1].plot(x_acc, train_acc_list[1:], label='train')
axs[1].set_title('Accuracy vs Iteration')
axs[1].set_xlabel('Iteration')
axs[1].set_ylabel('Accuracy')
axs[1].legend()

plt.tight_layout()
plt.show()

In [ ]:
def train_conv(model, optimizer, epochs=1, start_epoch=0):
    model = model.to(device=device)  # move the model parameters to CPU/GPU
    loss_list = []
    val_acc_list = []
    train_acc_list = []
    for e in range(epochs):
        print(f"epoch: {e+1}/{epochs}")
        for t, (x, y) in enumerate(loader_train):
            x = x.unsqueeze(dim = 1)
         
            model.train()  # put model to training mode
            x = x.to(device=device, dtype=dtype)  # move to device, e.g. GPU
            y = y.to(device=device, dtype=torch.long)
        
            scores = model(x)
            loss = F.cross_entropy(scores, y)

            # Zero out all of the gradients for the variables which the optimizer
            # will update.
            optimizer.zero_grad()

            # This is the backwards pass: compute the gradient of the loss with
            # respect to each  parameter of the model.
            loss.backward()

            # Actually update the parameters of the model using the gradients
            # computed by the backwards pass.
            optimizer.step()

            if t % print_every == 0:
                print('Iteration %d, loss = %.4f' % (t, loss.item()))
                loss_list.append(loss.item())
                print("Train Accuracy:")
                train_acc = check_accuracy_conv(loader_train, model)
                print("Validation Accuracy:")
                val_acc = check_accuracy_conv(loader_val, model)
                train_acc_list.append(train_acc)
                val_acc_list.append(val_acc)
                np.savetxt("train.txt", np.array(train_acc_list))
                np.savetxt("val_acc.txt", np.array(val_acc_list))
                np.savetxt("losslist.txt", np.array(loss_list))
                torch.save(model.state_dict(), 'model_weights_orig_conv.pth')
                print("model saved")
                print()
                

    return loss_list, train_acc_list, val_acc_list
            
def check_accuracy_conv(loader, model):
    num_correct = 0
    num_samples = 0
    model.eval()  # set model to evaluation mode
    with torch.no_grad():
        for x, y in loader:
            x = x.unsqueeze(dim = 1)
            x = x.to(device=device, dtype=dtype)  # move to device, e.g. GPU
            y = y.to(device=device, dtype=torch.long)
            scores = model(x)
            preds = torch.argmax(scores,dim=1)
            num_correct += (preds == y).sum()
            num_samples += preds.size(0)
        acc = float(num_correct) / num_samples
        print('Got %d / %d correct (%.2f)' % (num_correct, num_samples, 100 * acc))
    return acc

In [ ]:
conv_model = nn.Sequential(
    nn.Conv1d(1, 8, 9, padding = 'same'),
    nn.ReLU(),
    nn.MaxPool1d(2, stride=2),
    nn.Conv1d(8,16,9, padding = 'same'),
    nn.ReLU(),
    nn.MaxPool1d(2, stride=2),
    nn.Conv1d(16,32,9,  padding = 'same'),
    nn.ReLU(),
    nn.MaxPool1d(2, stride=2),
    nn.Conv1d(32,32,9,  padding = 'same'),
    nn.ReLU(),
    nn.MaxPool1d(2, stride=2),
    nn.Conv1d(32,32,9,  padding = 'same'),
    nn.ReLU(),
    nn.MaxPool1d(2, stride=2),
    nn.Flatten(),
    nn.Linear(64 * 2930, 22)
)

learning_rate = 5e-5
optimizer = optim.SGD(conv_model.parameters(), lr=learning_rate, momentum=0.9, nesterov=True)

# Start training
loss_list, train_acc_list, val_acc_list = train_conv(conv_model, optimizer, epochs=100)